# 티웨이항공 사전 데이터 EDA - for Customer

- 일자: 2022.08.22. ~ 08.31.
- 작업자: 전유빈, 김상욱

In [34]:
import pandas as pd
import sagemaker
import matplotlib.pyplot as plt

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

bucket = 'poc-2209-twayairport-dp'

booking_df = pd.read_csv(f's3://{bucket}/data/original/예약_20220822.csv')
coupon_df = pd.read_csv(f's3://{bucket}/data/original/쿠폰_20220822.csv')
flight_df = pd.read_csv(f's3://{bucket}/data/original/비행편정보_20220822.csv')
weather_df = pd.read_csv(f's3://{bucket}/data/etc/OBS_ASOS_TIM_20220831155407.csv', encoding='CP949')

# 데이터 크기, 형태 체크
print('1. 총 데이터 (건수, Feature수)')
print(f' - 예약데이터:   {booking_df.shape}')
print(f' - 쿠폰데이터:   {coupon_df.shape}')
print(f' - 비행편데이터: {flight_df.shape}')

# 컬럼명 변경은 핵심적으로 사용할 coupon_df 기준으로 변경
booking_df.rename(columns={'flight_number': 'flightno',
                           'flight_departure_date': 'flightdate',
                           'capture_date': 'issuedate',
                           'airline_code': 'carrier',
                           'fare_class': 'bookingclass',
                           'cabin_code': 'cabinclass'}, inplace=True)

flight_df.rename(columns={'flight_number': 'flightno',
                          'flight_departure_date': 'flightdate',
                          'cabin_class': 'cabinclass'}, inplace=True)

# 날짜 데이터 형변환
booking_df['flightdate'] = pd.to_datetime(booking_df['flightdate'], format='%Y-%m-%d')
booking_df['issuedate'] = pd.to_datetime(booking_df['issuedate'], format='%Y-%m-%d')
coupon_df['flightdate'] = pd.to_datetime(coupon_df['flightdate'], format='%Y%m%d')
coupon_df['issuedate'] = pd.to_datetime(coupon_df['issuedate'], format='%Y%m%d')
flight_df['flightdate'] = pd.to_datetime(flight_df['flightdate'], format='%Y-%m-%d')
flight_df['leg_departure_date_time'] = pd.to_datetime(flight_df['leg_departure_date_time'])
flight_df['leg_arrival_date_time'] = pd.to_datetime(flight_df['leg_arrival_date_time'])

flightno_list = flight_df['flightno'].unique()

# 날씨 데이터 가공
weather_df['강수량(mm)'].fillna(0, inplace=True)
weather_df['일시'] = pd.to_datetime(weather_df['일시'])
weather_df['date'] = weather_df['일시'].dt.date

date_list = list(weather_df['date'].unique())
temp_min = list(weather_df.groupby(['date'])['기온(°C)'].min())
temp_max = list(weather_df.groupby(['date'])['기온(°C)'].max())
temp_avg = list(weather_df.groupby(['date'])['기온(°C)'].mean())
precipitation_total = list(weather_df.groupby(['date'])['강수량(mm)'].sum())
cloudiness = list(weather_df.groupby(['date'])['전운량(10분위)'].mean())

new_weather_df = pd.DataFrame({
                    'date': date_list,
                    'temp_min': temp_min,
                    'temp_max': temp_max,
                    'temp_avg': temp_avg,
                    'precipitation_total': precipitation_total,
                    'cloudiness': cloudiness
                })
new_weather_df['date'] = pd.to_datetime(new_weather_df['date'])

# 누적판매좌석 추출
seats_df = pd.DataFrame({'total_sold_seats' : booking_df.groupby(["flightno", "issuedate", 'cabinclass'])['sold_seats'].sum()}).reset_index()

# 항공편데이터와 merge
seats_df = pd.merge(seats_df, flight_df[['flightno', 'cabinclass', 'cabin_authorised_cap']], on=['flightno', 'cabinclass'], how='left')

# 일일 판매좌석 테이블 생성
new_seats_df = pd.DataFrame()

for flightno in sorted(flightno_list):
    temp_df = seats_df[(seats_df['flightno'] == flightno) & (seats_df['cabinclass'] == 'Y')]
    temp_df['sold_seats'] = temp_df['total_sold_seats'] - temp_df['total_sold_seats'].shift().fillna(0)

    new_seats_df = pd.concat([new_seats_df, temp_df], axis=0)

# 쿠폰 데이터에서 추가로 얻을 수 있는 값.
coupon_df['remaindate'] = (coupon_df['flightdate'] - coupon_df['issuedate']).dt.days
coupon_df['issueweekday'] = coupon_df['issuedate'].dt.weekday
coupon_df['fare_1000'] = coupon_df['fare'] // 1000 * 1000

# 쿠폰데이터 기준 테이블 합치기
flight_columns = ['flightno', 'cabinclass', 'cabin_authorised_cap', 'leg_departure_date_time', 'leg_arrival_date_time']
seats_columns = ['flightno', 'issuedate', 'cabinclass', 'sold_seats', 'total_sold_seats']

new_coupon_df = pd.merge(coupon_df, flight_df[flight_columns], on=['flightno', 'cabinclass'], how='left')
new_coupon_df = pd.merge(new_coupon_df, new_seats_df[seats_columns], on=['flightno', 'issuedate', 'cabinclass'], how='left')
new_coupon_df = pd.merge(new_coupon_df, new_weather_df, left_on=['issuedate'], right_on=['date'], how='left')

new_coupon_df = new_coupon_df.dropna()

new_coupon_df['departure_hour'] = new_coupon_df['leg_departure_date_time'].dt.hour
new_coupon_df['departure_minute'] = new_coupon_df['leg_departure_date_time'].dt.minute
new_coupon_df['departure_time_format_min'] = new_coupon_df['departure_hour'] * 60 + new_coupon_df['departure_minute']

1. 총 데이터 (건수, Feature수)
 - 예약데이터:   (151848, 15)
 - 쿠폰데이터:   (6727, 15)
 - 비행편데이터: (34, 7)


In [35]:
# 이상치 제거
df = new_coupon_df.copy()

# 비즈니스석 제외
df = df[df['cabinclass'] == 'Y']

# bookingclass 특수값 제거
business = ['F', 'C', 'R', 'CC', 'CG', 'C1', 'C2'] # 이코너미만 체크할 것
economic = ['Y', 'Z', 'W', 'V', 'T', 'S', 'P', 'O', 'N', 'M', 'L', 'I', 'H', 'J', 'E', 'D', 'B']
event = ['A', 'BB', 'BL', 'UU', 'U',  'G', 'Q', 'R', 'X'] # 정상가가 아님 고려대상 X

df = df[df['bookingclass'].isin(economic)]

# 이코노미, 특수값 제거 후에도 0원 짜리가 존재함
df = df[df['fare'] != 0]

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

target_y = ['fare']
feature_x = ['departure_hour', 'departure_minute', 'departure_time_format_min'
             , 'sold_seats', 'total_sold_seats', 'remaindate']

df['remaindate'] = df['remaindate'].astype('float')
df['departure_hour'] = df['departure_hour'].astype('float')
df['departure_minute'] = df['departure_minute'].astype('float')
df['departure_time_format_min'] = df['departure_time_format_min'].astype('float')

gimpo = df[df['startseg'] == 'GMP']
jeju = df[df['startseg'] == 'CJU']

gimpo_x = gimpo[feature_x]
gimpo_y = gimpo[target_y]

## Train/Test Split
train_x, test_x, train_y, test_y = train_test_split(gimpo_x, gimpo_y, test_size=0.2, random_state=42)

model = RandomForestRegressor(max_depth=10, random_state=42)
model.fit(train_x, train_y)

print('=' * 50)
## Train 점수
print('train score....')
print(f'coefficient of determination of the prediction: {model.score(train_x, train_y)}')
pred = model.predict(train_x)
RMSE = mean_squared_error(train_y, pred)**0.5
print(f'RMSE: {RMSE}')
print('=' * 50)
## Test 점수
print('test score....')
print(f'coefficient of determination of the prediction: {model.score(test_x, test_y)}')
pred = model.predict(test_x)
RMSE = mean_squared_error(test_y, pred)**0.5
print(f'RMSE: {RMSE}')


# feature importance
print('=' * 50)
print(f'각 Feature별 영향도 확인')
feature_importances = model.feature_importances_
for i in range(len(feature_x)):
    print(f"{feature_x[i]}: {feature_importances[i]}")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:25: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


train score....
coefficient of determination of the prediction: 0.9834859874869375
RMSE: 2133.8820978973195
test score....
coefficient of determination of the prediction: 0.9753403558924839
RMSE: 2618.1119442673385
각 Feature별 영향도 확인
departure_hour: 0.3280452359698354
departure_minute: 0.021102543518425162
departure_time_format_min: 0.3421630415060493
sold_seats: 0.013179399031447676
total_sold_seats: 0.21355963324819247
remaindate: 0.0819501467260498
